In [1]:
import numpy as np
import pandas as pd

In [2]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies.head(1)

In [ ]:
credits.head(1)

In [ ]:
credits.head(1)['cast'].values

In [ ]:
credits.shape,movies.shape

In [8]:
movies=movies.merge(credits,on='title')

In [ ]:
movies.columns

In [10]:
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.head(1)

In [ ]:
movies.isnull().sum()

In [14]:
movies.dropna(inplace=True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [176]:
# import json
# [x['name'] for x in json.loads((movies.iloc[0].genres))]

In [17]:
import ast

In [18]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
convert(movies.iloc[0].genres)

In [20]:
movies['genres']=movies['genres'].apply(convert)

In [ ]:
movies.head(1)

In [22]:
movies['keywords']=movies['keywords'].apply(convert)

In [ ]:
movies['crew'][0]

In [24]:
def convert3(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter!=3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [25]:
movies['cast']=movies['cast'].apply(convert3)

In [26]:
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L

In [27]:
movies['crew']=movies['crew'].apply(fetch_director)

In [ ]:
movies.head(1)

In [29]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [30]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(' ','') for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(' ','') for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(' ','') for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(' ','') for i in x])

In [31]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [32]:
new_df=movies[['movie_id','title','tags']]

In [ ]:
new_df.head(1)

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:' '.join(x))

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df['tags'][0]

In [37]:
import nltk

In [38]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [39]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return ' '.join(y)

In [ ]:
new_df['tags']=new_df['tags'].apply(stem)

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [42]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors

In [ ]:
for i in cv.get_feature_names_out():
    print(i)

In [45]:
from sklearn.metrics.pairwise import cosine_similarity

In [46]:
similarity=cosine_similarity(vectors)

In [ ]:
similarity[1]

In [ ]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])

In [50]:
def recommend(movie):
    movie_index=new_df[new_df['title']==movie].index[0]
    distances=similarity[movie_index]
    movie_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movie_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('Avatar')

In [52]:
import pickle

In [211]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [212]:
pickle.dump(similarity,open('similarity.pkl','wb'))